In [2]:
import os
import sys

spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()

import pyspark

number_cores = 4
memory_gb = 8
conf = (pyspark.SparkConf().
        setMaster('local[{}]'.format(number_cores)).
        set('spark.driver.memory', '{}g'.format(memory_gb)))
sc = pyspark.SparkContext(conf=conf)

### Challenge 2

- Study the genome tags and answer the following question. How do user perceive and distinguish sub-genres? For example, what is the difference between action and action comedy?

### Examine the following
- genome-scores.csv
- genome-tags.csv
- tags.csv
- README.txt

In [7]:
#genome = sc.textFile("./ml-latest/genome-scores.csv")
!cat ./ml-latest/README.txt

Summary

This dataset (ml-latest) describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. It contains 27753444 ratings and 1108997 tag applications across 58098 movies. These data were created by 283228 users between January 09, 1995 and September 26, 2018. This dataset was generated on September 26, 2018.

Users were selected at random for inclusion. All selected users had rated at least 1 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in the files `genome-scores.csv`, `genome-tags.csv`, `links.csv`, `movies.csv`, `ratings.csv` and `tags.csv`. More details about the contents and use of all these files follows.

This is a *development* dataset. As such, it may change over time and is not an appropriate dataset for shared research results. See available *benchmark* datasets if that is your intent.

This and other GroupLe

In [8]:
!head ./ml-latest/genome-scores.csv

In [12]:
!head -20 ./ml-latest/genome-tags.csv

In [30]:
movies = sc.textFile('./ml-latest/movies.csv')
movies_header = movies.first()

indexed = movies.filter( lambda x: x != movies_header ).map( lambda x: (x.split(',')[0], x.split(',')[2].split('|')))
indexed.take(10)

[('1', ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']),
 ('2', ['Adventure', 'Children', 'Fantasy']),
 ('3', ['Comedy', 'Romance']),
 ('4', ['Comedy', 'Drama', 'Romance']),
 ('5', ['Comedy']),
 ('6', ['Action', 'Crime', 'Thriller']),
 ('7', ['Comedy', 'Romance']),
 ('8', ['Adventure', 'Children']),
 ('9', ['Action']),
 ('10', ['Action', 'Adventure', 'Thriller'])]

In [31]:
action_comedies = indexed.filter( lambda x: ('Action' in x[1]) and ('Comedy' in x[1]) )
action_comedies.take(3)

[('20', ['Action', 'Comedy', 'Crime', 'Drama', 'Thriller']),
 ('70', ['Action', 'Comedy', 'Horror', 'Thriller']),
 ('112', ['Action', 'Adventure', 'Comedy', 'Crime'])]

We want to:
- Build subgenre combination (just 2 for now)
- Associate these subgenres with the tags and tag scores

In [29]:
('2', 'Adventure-Children')
('2', 'Adventure-Fantasy')
('2', 'Children-Fantasy')

('2', ['Adventure-Children','Adventure-Fantasy','Children-Fantasy'])

('2', ['Adventure-Children', 'Adventure-Fantasy', 'Children-Fantasy'])

In [40]:
import itertools

def parse_subgenre( line ):
    # (x.split(',')[0], x.split(',')[2].split('|'))
    entry = line.split(',')
    entry[2] = entry[2].split('|')
    if len(entry[2]) < 2:
        continue
    else:
        entry[2] = list(itertools.combinations( entry[2], 2 ))
    return (entry[0],entry[2])

SyntaxError: 'continue' not properly in loop (<ipython-input-40-6a03a1aaf8b3>, line 11)

In [38]:
movie_subgenres = movies.filter( lambda x: x != movies_header ).map( parse_subgenre )
movie_subgenres.take(5)

[('1',
  [('Adventure', 'Animation'),
   ('Adventure', 'Children'),
   ('Adventure', 'Comedy'),
   ('Adventure', 'Fantasy'),
   ('Animation', 'Children'),
   ('Animation', 'Comedy'),
   ('Animation', 'Fantasy'),
   ('Children', 'Comedy'),
   ('Children', 'Fantasy'),
   ('Comedy', 'Fantasy')]),
 ('2',
  [('Adventure', 'Children'),
   ('Adventure', 'Fantasy'),
   ('Children', 'Fantasy')]),
 ('3', [('Comedy', 'Romance')]),
 ('4', [('Comedy', 'Drama'), ('Comedy', 'Romance'), ('Drama', 'Romance')]),
 ('5', [])]